In [3]:
!pip install langgraph langchain --quiet

In [7]:
# LangGraphを使用するためのJupyter Notebookのコード例

# 必要なライブラリをインポート
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableConfig
import random

# Stateの定義
class State(TypedDict):
    value: str  # 状態を保持するためのフィールド

# ノードの定義
def start_node(state: State, config: RunnableConfig):
    # 初期状態を返すノード
    return {"value": "1"}

def node2(state: State, config: RunnableConfig):
    # 状態を更新するノード
    return {"value": "2"}

def node3(state: State, config: RunnableConfig):
    # 状態を更新する別のノード
    return {"value": "3"}

# グラフビルダーの初期化
graph_builder = StateGraph(State)

# ノードをグラフに追加
graph_builder.add_node("start_node", start_node)
graph_builder.add_node("node2", node2)
graph_builder.add_node("node3", node3)

# 終了ノードを追加
graph_builder.add_node("end_node", lambda state: {"value": state["value"]})

# グラフのエントリーポイントを設定
graph_builder.set_entry_point("start_node")

# ルーティング関数の定義
def routing(state: State, config: RunnableConfig) -> Literal["node2", "node3"]:
    # ランダムに次のノードを決定
    random_num = random.randint(0, 1)
    if random_num == 0:
        return "node2"
    else:
        return "node3"

# 条件付きエッジを追加
graph_builder.add_conditional_edges("start_node", routing)

# ノード間のエッジを追加
graph_builder.add_edge("node2", "end_node")
graph_builder.add_edge("node3", "end_node")

# グラフの終了ポイントを設定
graph_builder.set_finish_point("end_node")

# グラフをコンパイル
graph = graph_builder.compile()

# グラフを実行し、初期状態を渡す
result = graph.invoke({"value": ""}, debug=True)

# 実行結果を表示
print(result)

[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'value': ''}
[0:writes] Finished step 0 with writes to 1 channel:
- value -> ''
[1:tasks] Starting step 1 with 1 task:
- start_node -> {'value': ''}
[1:writes] Finished step 1 with writes to 1 channel:
- value -> '1'
[2:tasks] Starting step 2 with 1 task:
- node3 -> {'value': '1'}
[2:writes] Finished step 2 with writes to 1 channel:
- value -> '3'
[3:tasks] Starting step 3 with 1 task:
- end_node -> {'value': '3'}
[3:writes] Finished step 3 with writes to 1 channel:
- value -> '3'
{'value': '3'}
